In [1]:
import pandas as pd
import numpy as np

In [167]:
dataset = pd.read_csv(r"C:\Users\mt19237\Desktop\Untitled Folder\ASS_4\titanic.csv")

dataset.Fare = dataset.Fare.round()


In [168]:
new_data=dataset.drop(['Name'], axis = 1) 
print ("totaL row=",new_data.shape)



new_data_train=new_data[0:dataset.shape[0]-1]
data_test=new_data[dataset.shape[0]-1:dataset.shape[0]]

print("train data size=",new_data_train.shape,"test data size=",data_test.shape)
new_data_train.head()

for i in new_data_train.columns: 
    print(i) 

totaL row= (887, 7)
train data size= (886, 7) test data size= (1, 7)
Survived
Pclass
Sex
Age
Siblings/Spouses Aboard
Parents/Children Aboard
Fare


In [169]:
def entropy(target_column):
    elements,counts = np.unique(target_column,return_counts = True)
    #print ("element",elements,"counts",counts)
    #print(elements)
    #print(counts)
    entropy = np.sum([
        (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) 
        for i in range(len(elements))
    ])
    return entropy


In [170]:
ent_col={}
entropy(new_data['Age'])
#for i in new_data_train.columns:
#    ent_col[i]=entropy(new_data[i])
#print(ent_col)


5.785783274284454

In [171]:
def info_gain(data,col_attr,col_target):

    total_entropy = entropy(data[col_target])
    
    #entropy of the dataset
    element,counts= np.unique(data[col_attr],return_counts=True)

    w_entropy = np.sum([(counts[i]/np.sum(counts))*
                               entropy(data.where(data[col_attr]==element[i]).dropna()[col_attr]) 
                               for i in range(len(element))])

    inf_gain = total_entropy - w_entropy
    return inf_gain
       

In [172]:
info_col={}
max1=float('-inf')
for i in new_data_train.columns:
    if i != 'Survived':
        info_col[i]=info_gain(new_data_train,'Survived',i)
        if info_col[i]>=max1:
            max1=info_col[i]
            k=i
print(k)
print(info_col)

Age
{'Pclass': 1.4425737495773556, 'Sex': 0.9379379226529758, 'Age': 5.7861796800063425, 'Siblings/Spouses Aboard': 1.3429909595560678, 'Parents/Children Aboard': 1.1324734959621252, 'Fare': 5.097828597892285}


In [173]:
#max info
root_node=k


In [174]:
tree = {root_node:{}}
print(tree)
element,counts= np.unique(new_data_train[root_node],return_counts=True)
print(element,counts)


{'Age': {}}
[ 0.42  0.67  0.75  0.83  0.92  1.    2.    3.    4.    5.    6.    7.
  8.    9.   10.   11.   12.   13.   14.   14.5  15.   16.   17.   18.
 19.   20.   20.5  21.   22.   23.   23.5  24.   24.5  25.   26.   27.
 28.   28.5  29.   30.   30.5  31.   32.   32.5  33.   34.   34.5  35.
 36.   36.5  37.   38.   39.   40.   40.5  41.   42.   43.   44.   45.
 45.5  46.   47.   48.   49.   50.   51.   52.   53.   54.   55.   55.5
 56.   57.   58.   59.   60.   61.   62.   63.   64.   65.   66.   69.
 70.   70.5  71.   74.   80.  ] [ 1  1  2  2  1  7 11  7 11  6  3  5  6  8  2  4  2  2  7  1  5 20 16 36
 33 23  1 34 39 25  1 34  1 24 21 26 37  2 25 33  2 19 20  2 17 17  1 21
 23  1 12 12 18 18  2  9 17  7  9 14  2  6 10 12  8 10  7  6  1  9  3  1
  4  3  5  2  5  3  5  2  3  3  2  1  2  1  2  1  1]


In [175]:
other_col = [i for i in new_data_train.columns
                 if (i != root_node and i !='Survived')
            ]
print(other_col)
print(np.unique(new_data_train[root_node]))

['Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']
[ 0.42  0.67  0.75  0.83  0.92  1.    2.    3.    4.    5.    6.    7.
  8.    9.   10.   11.   12.   13.   14.   14.5  15.   16.   17.   18.
 19.   20.   20.5  21.   22.   23.   23.5  24.   24.5  25.   26.   27.
 28.   28.5  29.   30.   30.5  31.   32.   32.5  33.   34.   34.5  35.
 36.   36.5  37.   38.   39.   40.   40.5  41.   42.   43.   44.   45.
 45.5  46.   47.   48.   49.   50.   51.   52.   53.   54.   55.   55.5
 56.   57.   58.   59.   60.   61.   62.   63.   64.   65.   66.   69.
 70.   70.5  71.   74.   80.  ]


In [187]:
sub_train_data={}
for value in np.unique(np.unique(new_data_train[root_node])):
    sub_train_data[value] = new_data_train.where(new_data_train[root_node] == value).dropna()
    #print(sub_train_data[value].shape)

print(sub_train_data[1].columns)
info_col={}
max1=float('-inf')
k=[]
for i in sub_train_data[1].columns:
    if i != 'Survived':
        info_col[i]=info_gain(sub_train_data[1],'Survived',i)
        #print(info_col[i])
        if info_col[i]>=max1:
            max1=info_col[i]
            k=i
print(k)
sec_level_node1=max(info_col)
print(sec_level_node1)

Index(['Survived', 'Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare'],
      dtype='object')
Fare
Siblings/Spouses Aboard
